In [1]:
$ProjectDirectory = $Pwd.Path
if($ProjectDirectory -match "Jupyter-PowerShell$") {
    @"
# How to build the Jupyter PowerShell Kernel

Start in the Jupyter-PowerShell folder, and make sure there's no output from the last run
"@ | Write-Jupyter -Mimetype markdown
} else {
    "# This notebook only works in the Jupyter-PowerShell project folder" | 
        Write-Jupyter -Mimetype markdown
    Write-Error "You cannot build Jupyter from here"
    $ProjectDirectory = $Env:Temp
}

# How to build the Jupyter PowerShell Kernel

Start in the Jupyter-PowerShell folder, and make sure there's no output from the last run

In [2]:
rm (Join-Path $ProjectDirectory Output\Release) -recurse

## To build the project

We really just need to `dotnet restore` and `dotnet build` --although currently, we require a very specific version (2.0.0-preview2-006502) of the dotnet CLI tools because we depend on the latest PowerShell Core bits, and they're _utterly_ incompatible with anything else.

### Increment the version

The project files to use the new `--version-suffix` feature. To make sure your build isn't confused with an official one, you need to specify a suffix, like "-local-preview".

## To package the project

In order to ship something, we need to `publish` it -- this includes the `build` step, so we can just call it directly.

Once we've published, we need to package it with Chocolatey.

In [3]:
dotnet restore

Restoring packages for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Jupyter\Jupyter.csproj...
  Restoring packages for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\PowerShell-Kernel.csproj...
  Restore completed in 715.68 ms for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Jupyter\Jupyter.csproj.
  Restore completed in 1.26 sec for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\PowerShell-Kernel.csproj.

In [4]:
dotnet publish -f netcoreapp2.0 -c Release --version-suffix "-beta-5"

Microsoft (R) Build Engine version 15.3.388.41745 for .NET Core
Copyright (C) Microsoft Corporation. All rights reserved.

  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\Jupyter.dll
  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\publish\
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\PowerShell-Kernel.dll
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\publish\

In [5]:
dotnet publish -f net462 -c Release --version-suffix "-beta-5"

Microsoft (R) Build Engine version 15.3.388.41745 for .NET Core
Copyright (C) Microsoft Corporation. All rights reserved.

  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\Jupyter.dll
  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\publish\
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\PowerShell-Kernel.exe
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\publish\

In [6]:
# We're just trying to rename the folders so that we can hash them:
Move-Item Output\Release\net462\publish Output\Release\PowerShell-Full
Move-Item Output\Release\netcoreapp2.0\publish Output\Release\PowerShell-Core
Copy-Item tools Output\Release -Recurse
Remove-Item Output\Release\net462 -Recurse
Remove-Item Output\Release\netcoreapp2.0 -Recurse

In [7]:
# Now generate the file catalog
New-FileCatalog -CatalogFilePath Output\Release\tools\Jupyter-PowerShell.cat -Path Output\Release\

Mode                LastWriteTime         Length Name                                                                   
----                -------------         ------ ----                                                                   
-a----        7/17/2017  12:53 AM         103522 Jupyter-PowerShell.cat

In [8]:
# Maybe sign the catalog
if(Get-Module Authenticode -List) {
    Authenticode\Set-AuthenticodeSignature Output\Release\tools\Jupyter-PowerShell.cat
}

Directory: C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\tools


SignerCertificate                         Status                                  Path                                  
-----------------                         ------                                  ----                                  
DC8A5C5FAFFBCFFE7F60552B49ED1A0DDC145482  Valid                                   Jupyter-PowerShell.cat

In [9]:
C:\ProgramData\chocolatey\choco.exe pack --outputdirectory Output\Release

Chocolatey v0.10.7
Attempting to build package from 'jupyter-powershell.nuspec'.
Successfully created package 'Output\Release\jupyter-powershell.1.0.0-beta-5.nupkg'